# Functional Connectivity Analysis
***

Now we have an idea of three important components to analyzing neuroimaging data:

1. Data manipulation
2. Cleaning and confound regression
3. Parcellation and signal extraction

In this notebook the goal is to integrate these 3 basic components and perform a full analysis of group data using **Seed-based Functional Connectivity (FC)**. 

Seed-based functional connectivity is essentially a result of performing correlational analysis on signals extracted from an ROI with voxels. Using this technique we can establish highly correlative regions in resting state data which form **functional networks**. 

This analysis forms a basis for many other sophisticated techniques in neuroimaging research. 

## Notebook Outline
***

The outline of the notebook is divided into two parts. The first part directly uses what you've learned and builds upon it to perform the final functional connectivity analysis on group data. 

The second part shows how we can use Nilearn's convenient wrapper functionality to perform the same task with *significantly less effort*. 

#### Part A: Manual computation 
1. Functional data cleaning and confound regression
2. Applying a parcellation onto the data
3. Seed-based functional connectivity analysis
4. Visualization

#### Part B: Using Nilearn's high-level features
1. Using NiftiMapsMasker to extract cleaned time-series
2. Performing seed-based functional connectivity analysis
3. Visualization

In [ ]:
import os
from nilearn import signal as sgl
from nilearn import image as img
from nilearn import plotting as plot
from nilearn import datasets
import nibabel as nib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sub = '70060'

In [ ]:
#Setting up directories
func_dir = '../data/func/'

#Pre-processed functional data in MNI space
func_file = 'sub-{}_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'.format(sub)
func_path = os.path.join(func_dir,func_file) 

#Confounds.tsv extracted using FMRIPREP 
confound_file = 'sub-{}_task-rest_bold_confounds.tsv'.format(sub)
confound_path = os.path.join(func_dir,confound_file) 

#MNI Brainmask outputted from FMRIPREP
mask_file = 'sub-{}_task-rest_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz'.format(sub)
mask_path = os.path.join(func_dir,mask_file) 

### Part A: Manual Computation of Seed-based functional connectivity
***

#### 1. Cleaning your functional data using filtering, dummy TR removal and confound regression
The first step to any functional analysis is to clean the data. We're going to make a python function that will handle confound cleaning for us:

We're also going to define a function to handle converting our confounds.tsv into a matrix

In [ ]:
#Refer to part_06 for code + explanation
def extract_confounds(confound_tsv,confounds,dt=True):
    '''
    Arguments:
        confound_tsv                    Full path to confounds.tsv
        confounds                       A list of confounder variables to extract
        dt                              Compute temporal derivatives [default = True]
        
    Outputs:
        confound_mat                    
    '''
    
    #Load in data using Pandas then extract relevant columns
    confound_df = pd.read_csv(confound_tsv,delimiter='\t') 
    confound_df = confound_df[confounds]
    
    #If using temporal derivatives 
    if dt:
        #For each column create a new column '<colname>_dt' containing the step-wise differences
        for col in confound_df.columns:
            confound_df['{}_dt'.format(col)] = confound_df[col].diff() 
    
    #Convert into a matrix of values (timepoints)x(variable)
    confound_mat = confound_df.values 
    
    #Return confound matrix
    return confound_mat
    

Let's clean our data!

In [ ]:
#Load functional image
func_img = img.load_img(func_path)
func_img = func_img.slicer[:,:,:,5:]

#Extract confounds
confounds = extract_confounds(confound_path,['X','Y','Z','RotX','RotY','RotZ','GlobalSignal'])
confounds = confounds[5:,:] 

#Clean functional image
clean_img = img.clean_img(func_img,confounds=confounds,low_pass=0.08,high_pass=0.009,t_r=2,
                         mask_img=mask_path) 

#### 2. Setting up the parcellation scheme

To apply a parcellation we'll have to specify a parcellation to use. 
For this analysis we'll be using [Craddock 2012 - 200 ROIs](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3838923/) as our parcellation scheme since it contains relatively simple ROIs that won't be too difficult to work with...

First we'll pull in our parcellation then extract the version containing 200 ROIs

In [ ]:
parcel_dir = '../resources/rois/' 
fetch_atlas = datasets.fetch_atlas_craddock_2012(parcel_dir) 

#Notice cc_atlas is 4D, where the 4th dimension contains different types of parcellations
cc_atlas = img.load_img(fetch_atlas['random']) 

#Index 19, contains the 200 ROI parcellation we want to use
cc200 = cc_atlas.slicer[:,:,:,19]

#Let's visualize it
plot.plot_roi(cc200,cmap='Paired')

Remember that in order to use the parcellation with our functional data it must have the same dimensions. It turns out that the parcellation schema has slightly different dimensions, so we need to resample: 

In [ ]:
#We use "nearest" to preserve the label values (a label of 2.42 for example, doesn't and shouldn't exist) 
resamp_cc200 = img.resample_to_img(cc200,clean_img,interpolation='nearest') 
print(resamp_cc200.shape)
print(clean_img.shape)

#### 3. Seed-based functional connectivity analysis
A seed-based functional connectivity is determined using the **mean time-series** from a single parcel (the seed) then correlating that with every other voxel in the brain. The idea behind this is to get an idea of how functionally connected other voxels are to a parcel. This analysis produces a **statistical map** which is then used in further analyses. 

In this section we will compute the mean time-series of a parcel from Craddock 200, compute the seed-based functional connectivity, then visualize on a glass-brain plot.
***


The first step is to select an ROI as the seed.

In [ ]:
#Select ROI and visualize
ROI = 174

#Visualize the ROI, we can do this by masking our all values not matching our ROI
roi_mask = img.math_img('a == {}'.format(ROI), a=resamp_cc200) 

#Apply the mask to the data
masked_resamp_cc200 = img.math_img('a*b',a=resamp_cc200,b=roi_mask) 

#Visualize
plot.plot_roi(masked_resamp_cc200) 

Next we extract the time-series of the voxels from the seed and compute the mean time-series across voxels in the seed

In [ ]:
#Recall that in the parcellation, each voxel is labelled with a number corresponding to a distinct parcel
#We want to extract parcels belonging to our ROI.

#Pull the voxels indices belonging to the ROI 
rcc200_data = resamp_cc200.get_data() 

#This yields a set of x,y,z coordinates corresponding to voxels belonging in the ROI 
roi = np.where(rcc200_data==ROI) 

#Now we'll take these coordinates and use them to pull the ROI time-series from our clean functional image
func_data = clean_img.get_data()

#This is now a matrix of size (voxels)x(timepoints) 
func_roi = func_data[roi]

#We want to take the average across voxels, we do this using numpy
mean_ts = np.mean(func_roi,axis=0)

The final step in seed-based connectivity is to compute correlations between the mean time-series and every other voxels in the brain. 

**BUT** 

If we compute correlations of our seed-regions to *all other voxels* in our functional image, we will be including background voxels (non-brain regions) in our correlational analysis as well. This won't necessarily pose a problem in our analysis if we make sure to apply a brain mask after our analysis. 

**BUT (again)**

Including non-brain regions in our correlation analysis will *slow us down significantly*. To speed up the analysis we will set non-brain correlations to 0, via *masking the voxels we correlate with*. This way we only perform the time-series correlation if and only if a particular voxel is a brain voxel

***

First we will load in the MNI space Mask image, then get the voxel coordinates where the mask is non-zero

In [ ]:
#Step 1: Load in mask data then get coordinates of non-zero voxels
#Since this mask is generated from the MNI space functional data, the dimensions should match
mask_img = img.load_img(mask_path) 
mask_data = mask_img.get_data() 

#Get mask region x,y,z coordinates
mask_x,mask_y,mask_z = np.where(mask_data == 1) 

The final output of our seed-based functional connectivity analysis will be a 3D volume, where each "voxel" will be assigned an $r$ correlation value with the seed mean timeseries (<code>mean_ts</code>). 
We'll initialize a 3D volume, where the values start off as zero, then assign $r$ values for each voxel, this will be our statistical map

In [ ]:
#Create an empty image for storing our statistical volume, needs to match x,y,z dimensions of our functional image
#Which is identical to the mask
stat_data = np.zeros_like(mask_data) 

Finally we loop through each of voxel coordinates contained within our mask, pull the time-series from the functional data and correlate with mean time series (<code>mean_ts</code>) we computed from our ROI 

In [ ]:
#Zip pairs up the x,y,z of each voxel, so that each loop's x,y,z will be associated with a particular voxel
for x,y,z in zip(mask_x,mask_y,mask_z): 
    
    #Grab the functional time-series for this particular voxel 
    rst_series = func_data[x,y,z,:]
    
    #Compute Pearson product-moment correlation ROI mean time-series and a single brain voxel
    r = np.corrcoef([mean_ts,rst_series])[0,1]
    
    #Assign r to voxel x,y,z in our statistical volume
    stat_data[x,y,z] = r

#### Part 4: Visulization of Results

To visualize our results, nilearn requires that our data be in nibabel Nifti1Image format. Recall that to create a Nifti1Image we need both our data (<code>stat_data</code>), and an affine matrix. We'll steal this affine matrix from our functional data!

In [ ]:
#Pull affine matrix from functional image
affine = func_img.affine

#Create statistical Nifti1Image
stat_img = nib.Nifti1Image(stat_data,affine) 

In [ ]:
plot.plot_img(stat_img)

In [ ]:
#3D Interactive viewer
plot.view_img(stat_img)